In [34]:
import os
import openai
from dotenv import load_dotenv
load_dotenv()

from typing import Dict, List, Any

from langchain.llms  import AzureOpenAI
from langchain.agents import create_pandas_dataframe_agent


API_KEY = os.getenv("OPENAI_API_KEY","").strip()
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

DEPLOYMENT_NAME="text-davinci-003"

llm = AzureOpenAI(
    openai_api_base=RESOURCE_ENDPOINT,
    deployment_name=DEPLOYMENT_NAME,
    openai_api_key=API_KEY,
    temperature=0
)

In [41]:
import pandas as pd

df = pd.read_csv('./data/titanic.csv')
df = df.drop('Name', axis=1)
agent = create_pandas_dataframe_agent(llm, df, verbose=True)

df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,male,26.0,0,0,111369,30.0000,C148,C


In [40]:
agent.run("Qual o quantidade de pessoas por classe? Quantos desses sobreviveram? Qual o percentual?")



> Entering new AgentExecutor chain...
Thought: I need to group the data by class and count the number of passengers and survivors
Action: python_repl_ast
Action Input: df.groupby('Pclass')['PassengerId'].count()
Observation: Pclass
1    216
2    184
3    491
Name: PassengerId, dtype: int64
Thought: I need to count the number of survivors
Action: python_repl_ast
Action Input: df.groupby('Pclass')['Survived'].sum()
Observation: Pclass
1    136
2     87
3    119
Name: Survived, dtype: int64
Thought: I need to calculate the percentage of survivors
Action: python_repl_ast
Action Input: df.groupby('Pclass')['Survived'].mean()
Observation: Pclass
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64
Thought: I now know the final answer
Final Answer: Na classe 1, havia 216 pessoas, das quais 136 sobreviveram, o que representa um percentual de 62,96%. Na classe 2, havia 184 pessoas, das quais 87 sobreviveram, o que representa um percentual de 47,28%. Na classe 3, havia 491 

'Na classe 1, havia 216 pessoas, das quais 136 sobreviveram, o que representa um percentual de 62,96%. Na classe 2, havia 184 pessoas, das quais 87 sobreviveram, o que representa um percentual de 47,28%. Na classe 3, havia 491 pessoas, das quais 119 sobreviveram, o que representa um percentual de 24,23%.'

In [42]:
agent.run("Qual correlação entre preço de passagem e sobreviventes?")



> Entering new AgentExecutor chain...
Thought: Devemos verificar a correlação entre as colunas Fare e Survived.
Action: python_repl_ast
Action Input: df[['Fare', 'Survived']].corr()
Observation:               Fare  Survived
Fare      1.000000  0.257307
Survived  0.257307  1.000000
Thought: Agora eu sei a resposta final.
Final Answer: A correlação entre preço de passagem e sobreviventes é de 0,257307.

> Finished chain.


'A correlação entre preço de passagem e sobreviventes é de 0,257307.'

In [43]:
agent.run("Adicione uma nova coluna com categoria de idade")



> Entering new AgentExecutor chain...
Thought: I need to create a new column and assign a category to each row
Action: python_repl_ast
Action Input: df['Age_Category'] = pd.cut(df['Age'], bins=[0, 18, 25, 40, 60, np.inf], labels=['Child', 'Young Adult', 'Adult', 'Middle Age', 'Senior'])
Observation: name 'pd' is not defined
Thought: I need to import pandas
Action: python_repl_ast
Action Input: import pandas as pd
Observation: 
Thought: Now I can create the new column
Action: python_repl_ast
Action Input: df['Age_Category'] = pd.cut(df['Age'], bins=[0, 18, 25, 40, 60, np.inf], labels=['Child', 'Young Adult', 'Adult', 'Middle Age', 'Senior'])
Observation: name 'np' is not defined
Thought: I need to import numpy
Action: python_repl_ast
Action Input: import numpy as np
Observation: 
Thought: Now I can create the new column
Action: python_repl_ast
Action Input: df['Age_Category'] = pd.cut(df['Age'], bins=[0, 18, 25, 40, 60, np.inf], labels=['Child', 'Young Adult', 'Adult', 'Middle Age', '

"The dataframe now has a new column called 'Age_Category' which contains the categories of age for each row."

In [44]:
df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_Category
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,Young Adult
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C,Adult
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Adult
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S,Adult
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S,Adult
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,211536,13.0000,NaN,S,Adult
887,888,1,1,female,19.0,0,0,112053,30.0000,B42,S,Young Adult
888,889,0,3,female,NaN,1,2,W./C. 6607,23.4500,NaN,S,NaN
889,890,1,1,male,26.0,0,0,111369,30.0000,C148,C,Adult


In [45]:
agent.run("Qual o valor total da vendas?")



> Entering new AgentExecutor chain...
Thought: I need to calculate the total sales value
Action: python_repl_ast
Action Input: df['Fare'].sum()
Observation: 28693.9493
Thought: I now know the final answer
Final Answer: O valor total das vendas é 28693.9493.

> Finished chain.


'O valor total das vendas é 28693.9493.'